In [2]:
import pandas as pd
import os
import matplotlib.pyplot as plt

# --- 1. Load the Merged Data ---
DATA_PATH = os.path.join("..", "data", "processed", "master_dataset_india.csv")
try:
    df_raw = pd.read_csv(DATA_PATH)
    print("✅ Raw merged data loaded successfully!")
except FileNotFoundError:
    print(f"❌ ERROR: File not found at {DATA_PATH}")
    raise SystemExit("Stopping script - master dataset not found.")

# --- 2. Aggregate Emissions by Year ---
print("\n--- Aggregating data by year ---")
id_confounder_policy_cols = [
    'Country_Name', 'Year', 'GDP per capita (constant 2015 US$)',
    'Industry (including construction), value added (% of GDP)',
    'Population, total',
    'Renewable energy consumption (% of total final energy consumption)',
    'policy_NAPCC_active'
]
pollutant_cols = [col for col in df_raw.columns if col.startswith(('EDGAR_', 'HCB_', 'PAH_', 'PCB_', 'PCDD_'))]
agg_dict = {poll_col: 'sum' for poll_col in pollutant_cols}
agg_dict.update({conf_col: 'first' for conf_col in id_confounder_policy_cols if conf_col not in ['Year']})
df_agg = df_raw.groupby('Year').agg(agg_dict).reset_index()
if 'Country_Name' not in df_agg.columns:
     df_agg['Country_Name'] = "India"
print("✅ Data aggregated by year.")

# --- 3. Display Info ---
df = df_agg.copy()
print("\n--- Aggregated DataFrame Info ---")
df.info()
print("\n--- Aggregated First 5 Rows ---")
print(df.head())

✅ Raw merged data loaded successfully!

--- Aggregating data by year ---
✅ Data aggregated by year.

--- Aggregated DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 23 columns):
 #   Column                                                              Non-Null Count  Dtype  
---  ------                                                              --------------  -----  
 0   Year                                                                53 non-null     int64  
 1   EDGAR_BC_1970_2022                                                  53 non-null     float64
 2   EDGAR_CO_1970_2022                                                  53 non-null     float64
 3   EDGAR_NH3_1970_2022                                                 53 non-null     float64
 4   EDGAR_NMVOC_1970_2022                                               53 non-null     float64
 5   EDGAR_NOx_1970_2022                                                 53 non-nul

In [3]:
print("--- Preparing data for DoWhy ---")
df_model = df.copy()
df_model = df_model.rename(columns={
    # 'EDGAR_PM2': 'pm25_emissions', # We won't rename specific pollutants now
    'policy_NAPCC_active': 'treatment',
    'GDP per capita (constant 2015 US$)': 'confounder_gdp',
    'Industry (including construction), value added (% of GDP)': 'confounder_industry_pct',
    'Population, total': 'confounder_population',
    'Renewable energy consumption (% of total final energy consumption)': 'confounder_renewables_pct'
})

# --- Handle Missing Data ---
missing_before = df_model.isnull().sum().sum()
df_model = df_model.ffill().bfill() # Combined ffill and bfill
missing_after = df_model.isnull().sum().sum()
print(f"Handled missing values. Missing count before: {missing_before}, after: {missing_after}")

print("\n--- Cleaned DataFrame for Model ---")
print(df_model.head())
print("\n--- Data Types ---")
print(df_model.info())

--- Preparing data for DoWhy ---
Handled missing values. Missing count before: 21, after: 0

--- Cleaned DataFrame for Model ---
   Year  EDGAR_BC_1970_2022  EDGAR_CO_1970_2022  EDGAR_NH3_1970_2022  \
0  1970           12.342949          917.397964            85.301905   
1  1971           12.287840          914.255856            85.860458   
2  1972           12.577959          922.258560            86.817770   
3  1973           13.086214          955.345018            87.880199   
4  1974           13.738303          966.594063            91.056906   

   EDGAR_NMVOC_1970_2022  EDGAR_NOx_1970_2022  EDGAR_OC_1970_2022  \
0             169.400080            88.786524           56.000582   
1             169.459928            88.487067           55.403164   
2             171.479649            89.046459           55.310541   
3             175.757650            89.801435           57.065146   
4             178.161069            93.957285           57.290451   

   EDGAR_PM10_1970_2022

In [4]:
import pandas as pd
from dowhy import CausalModel
import logging
import warnings

# Optional: Suppress warnings
warnings.filterwarnings("ignore", category=FutureWarning)
logging.getLogger("dowhy").setLevel(logging.ERROR) # Only show errors from DoWhy

# --- Define Model Components ---
common_causes = [
    'confounder_gdp',
    'confounder_industry_pct',
    'confounder_population',
    'confounder_renewables_pct',
    'Year'
]

# --- Identify Pollutant Columns in df_model ---
pollutant_cols_to_analyze = [col for col in df_model.columns if
                             col.startswith(('EDGAR_', 'HCB_', 'PAH_', 'PCB_', 'PCDD_'))
                            ]

print(f"--- Analyzing Causal Effect for {len(pollutant_cols_to_analyze)} Pollutants ---")
print(pollutant_cols_to_analyze)

results = []

# --- Loop through each pollutant ---
for pollutant in pollutant_cols_to_analyze:
    print(f"\n--- Processing: {pollutant} ---")
    ate_value = None
    p_value_ate = None
    p_value_placebo = None

    try:
        # Check for zero variance
        if df_model[pollutant].nunique(dropna=False) <= 1:
             print(f"  ⚠️ SKIPPED: '{pollutant}' has zero or only one unique value.")
             results.append({'Pollutant': pollutant, 'ATE': None, 'P_Value_ATE': None, 'P_Value_Placebo': None})
             continue

        # 1. Define Model
        model_poll = CausalModel(
            data=df_model, treatment='treatment', outcome=pollutant,
            common_causes=common_causes
        )
        # 2. Identify
        identified_estimand_poll = model_poll.identify_effect(proceed_when_unidentifiable=True)
        # 3. Estimate
        estimate_poll = model_poll.estimate_effect(
            identified_estimand_poll, method_name="backdoor.linear_regression",
            test_significance=True
        )
        ate_value = estimate_poll.value
        sig_test = estimate_poll.test_stat_significance()
        p_value_ate = float('nan')
        if sig_test and 'p_value' in sig_test and len(sig_test['p_value']) > 0:
             pval_candidate = sig_test['p_value'][0]
             if pd.notna(pval_candidate): p_value_ate = float(pval_candidate)
        print(f"  ATE: {ate_value:.3f}, p-value: {p_value_ate:.4f}")

        # 4. Refute (Attempting access via refutation_result)
        p_value_placebo = float('nan')
        refute_placebo_poll = None
        try:
            refute_placebo_poll = model_poll.refute_estimate(
                identified_estimand_poll, estimate_poll,
                method_name="placebo_treatment_refuter",
                placebo_type="permute", num_simulations=50
            )
            # *** UPDATED ACCESS ATTEMPT ***
            if refute_placebo_poll is not None and hasattr(refute_placebo_poll, 'refutation_result'):
                ref_result = refute_placebo_poll.refutation_result
                # Check if it's a dictionary and contains 'p_value'
                if isinstance(ref_result, dict) and 'p_value' in ref_result:
                    pval_candidate = ref_result['p_value']
                    if pd.notna(pval_candidate): p_value_placebo = float(pval_candidate)
                    else: print(f"  WARNING: Placebo p-value inside refutation_result is NaN/None for {pollutant}.")
                else:
                    # If not a dict or no p_value key, print for debugging
                    print(f"  DEBUG: 'p_value' key NOT FOUND in refutation_result for {pollutant}.")
                    print(f"  DEBUG: refutation_result content: {ref_result}")
                    print(f"  DEBUG: refutation_result type: {type(ref_result)}")
            elif refute_placebo_poll is not None:
                print(f"  DEBUG: Refuter object lacks 'refutation_result' attribute for {pollutant}.")
            else:
                print(f"  WARNING: Placebo refutation returned None for {pollutant}.")
            # *** END UPDATED BLOCK ***
        except Exception as refute_error:
            print(f"  ERROR during placebo refutation for {pollutant}: {refute_error}")

        # Print placebo p-value safely
        if pd.isna(p_value_placebo): print(f"  Placebo Refutation p-value: NaN")
        else: print(f"  Placebo Refutation p-value: {p_value_placebo:.4f}")

        results.append({
            'Pollutant': pollutant, 'ATE': ate_value,
            'P_Value_ATE': p_value_ate if not pd.isna(p_value_ate) else None,
            'P_Value_Placebo': p_value_placebo if not pd.isna(p_value_placebo) else None
        })
    except Exception as e:
        print(f"  ❌ Error processing {pollutant}: {e}")
        results.append({
            'Pollutant': pollutant, 'ATE': None, 'P_Value_ATE': None, 'P_Value_Placebo': None
        })

# --- Display Results ---
print("\n\n--- Summary of Causal Effects (Linear Regression) ---")
results_df = pd.DataFrame(results).round(4)
print(results_df.to_string())

significant_robust_results = results_df[
    (results_df['P_Value_ATE'].notna()) & (results_df['P_Value_ATE'] < 0.05) &
    (results_df['P_Value_Placebo'].notna()) & (results_df['P_Value_Placebo'] >= 0.05)
].copy()
print("\n--- Statistically Significant & Robust Effects (p_ATE < 0.05, p_Placebo >= 0.05) ---")
if significant_robust_results.empty: print("No statistically significant and robust effects found with this method.")
else: print(significant_robust_results.to_string())

--- Analyzing Causal Effect for 16 Pollutants ---
['EDGAR_BC_1970_2022', 'EDGAR_CO_1970_2022', 'EDGAR_NH3_1970_2022', 'EDGAR_NMVOC_1970_2022', 'EDGAR_NOx_1970_2022', 'EDGAR_OC_1970_2022', 'EDGAR_PM10_1970_2022', 'EDGAR_PM2', 'EDGAR_SO2_1970_2022', 'HCB_1970_2018', 'PAH_BaP_1970_2018', 'PAH_BbF_1970_2018', 'PAH_BkF_1970_2018', 'PAH_IcdP_1970_2018', 'PCB_1970_2018', 'PCDD_F_1970_2018']

--- Processing: EDGAR_BC_1970_2022 ---
  ATE: 2.451, p-value: 0.0187
  Placebo Refutation p-value: 0.4839

--- Processing: EDGAR_CO_1970_2022 ---
  ATE: 114.886, p-value: 0.0180
  Placebo Refutation p-value: 0.4328

--- Processing: EDGAR_NH3_1970_2022 ---
  ATE: 30.453, p-value: 0.0013
  Placebo Refutation p-value: 0.4545

--- Processing: EDGAR_NMVOC_1970_2022 ---
  ATE: 16.783, p-value: 0.0059
  Placebo Refutation p-value: 0.4994

--- Processing: EDGAR_NOx_1970_2022 ---
  ATE: 23.536, p-value: 0.0135
  Placebo Refutation p-value: 0.4889

--- Processing: EDGAR_OC_1970_2022 ---
  ATE: 10.149, p-value: 0.00

In [5]:
# # We now have the complete results, including the placebo test validation, for all 16 pollutants.

# # ## Analysis of Multi-Pollutant Results

# # Let's look at the final table showing effects that are both statistically significant (ATE p \< 0.05) and robust (Placebo p \>= 0.05):

# # ```
# # --- Statistically Significant & Robust Effects (p_ATE < 0.05, p_Placebo >= 0.05) ---
# #                Pollutant       ATE  P_Value_ATE  P_Value_Placebo
# # 0     EDGAR_BC_1970_2022    2.4514       0.0187           0.4163
# # 1     EDGAR_CO_1970_2022  114.8857       0.0180           0.4571
# # 2    EDGAR_NH3_1970_2022   30.4533       0.0013           0.4466
# # 3  EDGAR_NMVOC_1970_2022   16.7827       0.0059           0.4959
# # 4    EDGAR_NOx_1970_2022   23.5355       0.0135           0.4018
# # 5     EDGAR_OC_1970_2022   10.1487       0.0041           0.4693
# # 6   EDGAR_PM10_1970_2022   17.6442       0.0056           0.4825
# # 7              EDGAR_PM2   11.7729       0.0045           0.4403
# # 8    EDGAR_SO2_1970_2022   44.6982       0.0366           0.4469
# # ```

# # **Key Findings:**

# # 1.  **Consistent Positive Effect:** The counter-intuitive **positive** causal effect (ATE \> 0) holds true for **all 9 major air pollutants** 
# where the result was statistically significant and robust\! This includes PM2.5, PM10, SO2, NOx, CO, BC (Black Carbon), OC (Organic Carbon), 
# NMVOCs (Non-Methane Volatile Organic Compounds), and NH3 (Ammonia).
# # 2.  **Robustness Confirmed:** All these significant results passed the placebo test (P\_Value\_Placebo \>= 0.05), increasing our confidence 
# that these are not just random statistical flukes.
# # 3.  **No Significant Effect (or Negative Trend) for Others:** Pollutants like HCB, PAHs, PCB, PCDD/F showed either no statistically significant 
# effect (p \> 0.05) or a negative ATE that wasn't significant.

# # -----

# # ## Interpretation & Demo Narrative

# # This multi-pollutant analysis **strengthens our core finding** and makes the demo story even more compelling.

# # You can now confidently state:
# # "Our Causal Engine analyzed the impact of India's post-2008 climate policy period (NAPCC launch) on 16 different pollutants,
# controlling for economic growth, population, and other factors. We found a **consistent, statistically significant, and robust *positive* causal 
# effect** for major air pollutants like PM2.5, SO2, and NOx. This suggests that during the initial years of the NAPCC, emissions for these key 
# pollutants were **higher than expected** based on underlying economic and demographic trends alone. While counter-intuitive compared to the raw 
# data trends, this robust finding highlights the complexity of policy impact. Possible explanations include lagged policy effects, initial 
# implementation activities, or shifts in the economy not fully captured by our model, warranting further investigation into sector-specific data."

# # This demonstrates a powerful, nuanced analysis well beyond simple plotting.

In [15]:
pip install econml

Note: you may need to restart the kernel to use updated packages.


In [12]:
# --- [Corrected-V5] Standalone DML Check for ALL Pollutants ---
# This version fixes the "Cannot use a classifier" error by
# using a Regressor for model_t, which econml expects.

print("\n\n--- Running [Corrected-V5] DML Sensitivity Check for All Pollutants ---")

# --- 1. Imports ---
try:
    import econml
    import dowhy
    from dowhy import CausalModel
    import pandas as pd
    
    # Import the models we will instantiate
    from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
    from sklearn.linear_model import LinearRegression
    
    # Imports for suppressing warnings
    import warnings
    from sklearn.exceptions import DataConversionWarning
    
    print("Libraries imported successfully.")
except ImportError as e:
    print(f"❌ Failed to import libraries: {e}")
    print("   Please ensure 'dowhy', 'econml', and 'scikit-learn' are installed.")
    raise e
except NameError as e:
    print(f"❌ Failed to import libraries: {e}")
    print("   This can happen in some environments. Please install 'econml' and 'scikit-learn'.")
    raise e

# Suppress the noisy sklearn warnings
warnings.filterwarnings("ignore", category=DataConversionWarning)

# --- 2. Setup (Using variables from Cell 1) ---
try:
    POLLUTANT_COLS = [col for col in df_model.columns if
                      col.startswith(('EDGAR_', 'HCB_', 'PAH_', 'PCB_', 'PCDD_'))]
    TREATMENT_COL = 'treatment'
    COMMON_CAUSES_LIST = [
        'confounder_gdp',
        'confounder_industry_pct',
        'confounder_population',
        'confounder_renewables_pct',
        'Year'
    ]
    dml_ate_results = {}
    print(f"Testing for Treatment: '{TREATMENT_COL}'")
    print(f"Testing on {len(POLLUTANT_COLS)} Outcome(s)\n")

except NameError as e:
    print(f"❌ DML FAILED: Could not find 'df_model' or other variables from Cell 1.")
    print(f"   Error: {e}")
    print("   SOLUTION: Please make sure you have successfully run the *entire* cell above this one.")
    raise e

# --- 3. Loop Through Each Pollutant ---
for outcome in POLLUTANT_COLS:
    print(f"--- Processing DML for: {outcome} ---")
    
    try:
        if df_model[outcome].nunique(dropna=False) <= 1:
            print(f"  ⚠️ SKIPPED: '{outcome}' has zero or only one unique value.")
            dml_ate_results[outcome] = None
            continue

        # --- 3a. Define Model ---
        model = CausalModel(
            data=df_model,
            treatment=TREATMENT_COL,
            outcome=outcome,
            common_causes=COMMON_CAUSES_LIST
        )

        # --- 3b. Identify Estimand ---
        identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
        
        # --- 3c. Run DML Estimation ---
        print(f"Running DML estimation for {outcome}...")
        estimate_dml = model.estimate_effect(
            identified_estimand,
            method_name="backdoor.econml.dml.DML",
            control_value=0,
            treatment_value=1,
            target_units="ate",
            method_params={
                "init_params": {
                    # *** THIS IS THE FIX ***
                    # econml's DML expects a REGRESSOR for model_t,
                    # even when the treatment is binary.
                    'model_y': GradientBoostingRegressor(),
                    'model_t': GradientBoostingRegressor(), # Was GradientBoostingClassifier()
                    'model_final': LinearRegression()
                },
                "fit_params": {}
            }
        )
        
        ate_value = estimate_dml.value
        print(f"✅ {outcome} ATE (DML): {ate_value:.3f}")
        dml_ate_results[outcome] = ate_value

    except Exception as e:
        print(f"❌ DML estimation failed for {outcome}: {e}")
        dml_ate_results[outcome] = None 

# --- 4. Final Summary ---
print("\n\n--- DML SENSITIVITY CHECK: FINAL SUMMARY ---")
print("Estimated ATE (Average Treatment Effect) using DML:")
print(pd.Series(dml_ate_results, name="DML_ATE"))
print("-------------------------------------------------")



--- Running [Corrected-V5] DML Sensitivity Check for All Pollutants ---
Libraries imported successfully.
Testing for Treatment: 'treatment'
Testing on 16 Outcome(s)

--- Processing DML for: EDGAR_BC_1970_2022 ---
Running DML estimation for EDGAR_BC_1970_2022...


D:\Installed\MiniConda\envs\ai_env\Lib\site-packages\econml\dml\dml.py:192: UserWarning: The final model has a nonzero intercept for at least one outcome; it will be subtracted, but consider fitting a model without an intercept if possible.
  warn("The final model has a nonzero intercept for at least one outcome; "
D:\Installed\MiniConda\envs\ai_env\Lib\site-packages\econml\dml\dml.py:192: UserWarning: The final model has a nonzero intercept for at least one outcome; it will be subtracted, but consider fitting a model without an intercept if possible.
  warn("The final model has a nonzero intercept for at least one outcome; "


✅ EDGAR_BC_1970_2022 ATE (DML): 1.345
--- Processing DML for: EDGAR_CO_1970_2022 ---
Running DML estimation for EDGAR_CO_1970_2022...
✅ EDGAR_CO_1970_2022 ATE (DML): 54.054
--- Processing DML for: EDGAR_NH3_1970_2022 ---
Running DML estimation for EDGAR_NH3_1970_2022...


D:\Installed\MiniConda\envs\ai_env\Lib\site-packages\econml\dml\dml.py:192: UserWarning: The final model has a nonzero intercept for at least one outcome; it will be subtracted, but consider fitting a model without an intercept if possible.
  warn("The final model has a nonzero intercept for at least one outcome; "
D:\Installed\MiniConda\envs\ai_env\Lib\site-packages\econml\dml\dml.py:192: UserWarning: The final model has a nonzero intercept for at least one outcome; it will be subtracted, but consider fitting a model without an intercept if possible.
  warn("The final model has a nonzero intercept for at least one outcome; "


✅ EDGAR_NH3_1970_2022 ATE (DML): 15.897
--- Processing DML for: EDGAR_NMVOC_1970_2022 ---
Running DML estimation for EDGAR_NMVOC_1970_2022...
✅ EDGAR_NMVOC_1970_2022 ATE (DML): 14.679
--- Processing DML for: EDGAR_NOx_1970_2022 ---
Running DML estimation for EDGAR_NOx_1970_2022...


...

D:\Installed\MiniConda\envs\ai_env\Lib\site-packages\econml\dml\dml.py:192: UserWarning: The final model has a nonzero intercept for at least one outcome; it will be subtracted, but consider fitting a model without an intercept if possible.
  warn("The final model has a nonzero intercept for at least one outcome; "
D:\Installed\MiniConda\envs\ai_env\Lib\site-packages\econml\dml\dml.py:192: UserWarning: The final model has a nonzero intercept for at least one outcome; it will be subtracted, but consider fitting a model without an intercept if possible.
  warn("The final model has a nonzero intercept for at least one outcome; "


✅ PAH_BbF_1970_2018 ATE (DML): -4.952
--- Processing DML for: PAH_BkF_1970_2018 ---
Running DML estimation for PAH_BkF_1970_2018...
✅ PAH_BkF_1970_2018 ATE (DML): 0.002
--- Processing DML for: PAH_IcdP_1970_2018 ---
Running DML estimation for PAH_IcdP_1970_2018...


D:\Installed\MiniConda\envs\ai_env\Lib\site-packages\econml\dml\dml.py:192: UserWarning: The final model has a nonzero intercept for at least one outcome; it will be subtracted, but consider fitting a model without an intercept if possible.
  warn("The final model has a nonzero intercept for at least one outcome; "


✅ PAH_IcdP_1970_2018 ATE (DML): 0.004
--- Processing DML for: PCB_1970_2018 ---
Running DML estimation for PCB_1970_2018...
✅ PCB_1970_2018 ATE (DML): -0.000
--- Processing DML for: PCDD_F_1970_2018 ---
Running DML estimation for PCDD_F_1970_2018...
✅ PCDD_F_1970_2018 ATE (DML): 0.000


--- DML SENSITIVITY CHECK: FINAL SUMMARY ---
Estimated ATE (Average Treatment Effect) using DML:
EDGAR_BC_1970_2022       1.345020e+00
EDGAR_CO_1970_2022       5.405428e+01
EDGAR_NH3_1970_2022      1.589704e+01
EDGAR_NMVOC_1970_2022    1.467910e+01
EDGAR_NOx_1970_2022      2.122386e+01
EDGAR_OC_1970_2022       3.527460e+00
EDGAR_PM10_1970_2022     3.297734e+00
EDGAR_PM2                4.155887e+00
EDGAR_SO2_1970_2022      2.389342e+01
HCB_1970_2018           -4.624942e-07
PAH_BaP_1970_2018       -1.325670e-03
PAH_BbF_1970_2018       -4.952212e+00
PAH_BkF_1970_2018        2.083298e-03
PAH_IcdP_1970_2018       4.138445e-03
PCB_1970_2018           -2.254846e-05
PCDD_F_1970_2018         2.993316e-08
Name: D

D:\Installed\MiniConda\envs\ai_env\Lib\site-packages\econml\dml\dml.py:192: UserWarning: The final model has a nonzero intercept for at least one outcome; it will be subtracted, but consider fitting a model without an intercept if possible.
  warn("The final model has a nonzero intercept for at least one outcome; "
D:\Installed\MiniConda\envs\ai_env\Lib\site-packages\econml\dml\dml.py:192: UserWarning: The final model has a nonzero intercept for at least one outcome; it will be subtracted, but consider fitting a model without an intercept if possible.
  warn("The final model has a nonzero intercept for at least one outcome; "
